# NLP Project
### Group 6
- Ashwin R
- Suriya Shanmugha Sundaram
- Abhinav B
- Ganeshan M

## Imports


In [86]:
import os
import re
import argparse
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [85]:
import pandas as pd
from bs4 import BeautifulSoup
import fastai, torch

import nltk, string, re, spacy,unicodedata, random
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Dataset fetching and cleaning

In [ ]:
# Downloading datasets
!wget https://raw.githubusercontent.com/ashwinkey04/offensive-lang-nlp/main/test.csv;
!wget https://raw.githubusercontent.com/ashwinkey04/offensive-lang-nlp/main/train.csv;


--2022-03-24 16:10:31--  https://raw.githubusercontent.com/ashwinkey04/offensive-lang-nlp/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121252 (118K) [text/plain]
Saving to: ‘test.csv.2’

test.csv.2          100%[===================>] 118.41K  --.-KB/s    in 0.01s   

2022-03-24 16:10:31 (7.75 MB/s) - ‘test.csv.2’ saved [121252/121252]

--2022-03-24 16:10:31--  https://raw.githubusercontent.com/ashwinkey04/offensive-lang-nlp/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 537634 (525K) [text/plain]
S

In [82]:
df = pd.read_csv('train.csv', sep=',')
df_test = pd.read_csv('test.csv', sep=',')

df = df.dropna()
df = df.drop(['id'], axis = 1)

df_test = df_test.dropna()
df_test = df_test.drop(['id'], axis = 1)

In [ ]:
import re

# removing emojis
emoji = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags 
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f" 
    u"\u3030"
    "]+",
    flags=re.UNICODE,
)
emoticon_list = [
    "XD", "O:-)", ")-:", "(-:", ":-|", "@", "<3", "~,~", ":-B", "<l:0", "$_$", "8-)", ":-/", ":-(", ">:)", ":*)", "#-o", "*<:o)", "|-O", "~:0", ":/", "=-O", ":-#", ":-J", "=D", "B-)", ":'(", "*-*", ":)", "<><", ":o", "=()", "</3", "@~)~~~~", "//_^", "[¬º-°]¬", ":P", "^_^", ":-Q", "=^.^=", ":-*", "'=p'", ":->", "<:3)~", ":-E", ";-)", "=)", "#", ":S", "(*v*)", "O.o", "=8)", ":(", "<3", "(.V.)", ":>", ":-D", "=D", "=o", "(-}{-)", "X-(", ":-@", ":-\\", ".X-p", ":-&", ":-)(-:", "=/", "\\:D/", "{}", ":o3",
]

In [87]:
def preprocess_text(text):
    text = text.lower()
    text = text.strip()
    text = re.sub(r"http\S+", "", text, flags=re.MULTILINE)  # removing urls
    text = re.sub(r"www\S+", "", text, flags=re.MULTILINE)  # removing urls
    text = re.sub(r"&\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"\.{2,}", ".", text, flags=re.MULTILINE)  # continous dots
    text = re.sub(r"&", " and ", text)

    for i in [8221, 35, 36, 94, 8217]:
        text = text.replace(chr(i), "")
    for i in emoticon_list:
        text = text.replace(i, "")
    text = text.replace("\n", ".")
    text = text.replace("&amp", " ")
    text = text.replace("-", "")
    text = text.replace(";", "")
    text = text.replace("@", "")
    text = text.replace('[{}]'.format(string.punctuation), '') #removing punctuations

    # remove emoji
    text = emoji.sub(r"", text)
    punctuation = '!!"$%&()*+-/:;<=>?[\\]^_{|}~.'
    text = ''.join(ch for ch in text if ch not in set(punctuation))
    # text = text.translate(str.maketrans('', '', string.punctuation))
    text = BeautifulSoup(text, 'html.parser').get_text()
      # Stopword Removing
    tokenizer = ToktokTokenizer()
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    text = ' '.join(ch for ch in tokens)

    return text

In [88]:
def process_data(df):
    df["cleaned_text"] = df["text"].map(lambda x: preprocess_text(x))
    df["offensive"] = df["category"].map(lambda x: 1 if x.upper() == "OFF" else 0)

    return df

In [89]:
df = process_data(df)
df_test = process_data(df_test)

## Dataset preprocessing

* Lower case the Romanized words.
* Removing emojis and Emoticons
* Remove all special characters, numbers and punctuation
* Removing URLs, and user mentions

In [ ]:
import nltk
nltk.download('punkt')
df['tokenized_txt'] = df.apply(lambda row: nltk.word_tokenize(row['cleaned_text']), axis=1) #Tokenizing text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df['tokenized_txt']

0       [asha, apo, neenga, atha, government, ku, ther...
1       [bala, sundar, ayyo, sorry.antha, line, ah, ex...
2       [kalimuthu, ne, ena, lusa.yaaru, edhu, panaalu...
3       [1st, baby, ku, neat, ah, feed, panunga, plzz,...
4       [2012, il, vazhgiromnu, iruku, ithula, ,, apdi...
                              ...                        
3994    [yaroda, body, structure, semaya, irukum, ?, s...
3995    [yenda, naangala, politics, varom, nu, pala, v...
3996    [yepdithan, seruppala, adichalum, arasiyalvath...
3997    [user, paithiyam, ena, unga, vanthu, full, ah,...
3998    [rt, user, :, itha, vidaa, kevalam, veraa, iru...
Name: tokenized_txt, Length: 3999, dtype: object

In [ ]:
import spacy
from spacy.lang.en import English

nlp = English()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
text = df['text']
model = CountVectorizer(ngram_range = (1, 1), stop_words='english')
matrix = model.fit_transform(text).toarray()
df_output = pd.DataFrame(data = matrix, columns = model.get_feature_names())
df_output.T.tail(5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1,2,3,4,5,6,7,8,9,...,3989,3990,3991,3992,3993,3994,3995,3996,3997,3998
¾à,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
šalways,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
žˆ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
žˆthan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ˆà,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
